In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

C:\Users\pektu\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.feature_extraction.stop_words module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_extraction.text. Anything that cannot be imported from sklearn.feature_extraction.text is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
data = 'train.csv'
df = pd.read_csv(data)
df.head

<bound method NDFrame.head of        Unnamed: 0                                             review  label
0               0  I think they really let the quality of the DVD...      0
1               1  I'm sorry but this is just awful. I have told ...      0
2               2  The Japenese sense of pacing, editing and musi...      0
3               3  In the '60's/'70's, David Jason was renowned f...      1
4               4  "Hail The Woman" is one of the most moving fil...      1
...           ...                                                ...    ...
39995       39995  When you come across a gem of a movie like thi...      1
39996       39996  I don't often go out of my way to write commen...      0
39997       39997  This is an extremely silly and little seen fil...      0
39998       39998  Just saw the movie, and the scary thing was, t...      1
39999       39999  ...though for a film that seems to be trying t...      0

[40000 rows x 3 columns]>

In [3]:
df['review'][0]

"I think they really let the quality of the DVD production get away from them. I rented this DVD from 2 movie stores and the second time I finally got it to play on the 3rd DVD player I tried.<br /><br />Anyone else have this issue? It's really hard to give the film an un-biased review after going through such a hassle to play it. For one, I've never seen a Finnish horror film before so I was sort of bummed that the movie was done in English. Also since it's never made clear what is wrong with Sarah, she just came off as retarded and therefore I really just hoped someone would shoot her in the face and make all the horrific happenings go away."

In [4]:
df['review'][0].replace('<br />', ' ')

"I think they really let the quality of the DVD production get away from them. I rented this DVD from 2 movie stores and the second time I finally got it to play on the 3rd DVD player I tried.  Anyone else have this issue? It's really hard to give the film an un-biased review after going through such a hassle to play it. For one, I've never seen a Finnish horror film before so I was sort of bummed that the movie was done in English. Also since it's never made clear what is wrong with Sarah, she just came off as retarded and therefore I really just hoped someone would shoot her in the face and make all the horrific happenings go away."

In [9]:
preprocess_df(df)

,Unnamed: 0,review,label,excl_number,q_number,neg_number,pos_number,comm_number,double_excl_number
0,0,I think they really let the quality of the DVD...,0,0.000000,0.003371,0.0,0.0,0.003252,0.000000
1,1,I'm sorry but this is just awful. I have told ...,0,0.000000,0.003480,0.0,0.0,0.001678,0.000000
2,2,"The Japenese sense of pacing, editing and musi...",0,0.000000,0.000000,0.0,0.0,0.010511,0.000000
3,3,"In the '60's/'70's, David Jason was renowned f...",1,0.002572,0.000000,0.0,0.0,0.011702,0.000000
4,4,"""Hail The Woman"" is one of the most moving fil...",1,0.000000,0.000000,0.0,0.0,0.010711,0.000000
...,...,...,...,...,...,...,...,...,...
39995,39995,When you come across a gem of a movie like thi...,1,0.002259,0.000000,0.0,0.0,0.011423,0.000000
39996,39996,I don't often go out of my way to write commen...,0,0.001977,0.000000,0.0,0.0,0.005998,0.000000
39997,39997,This is an extremely silly and little seen fil...,0,0.003639,0.000000,0.0,0.0,0.000000,0.000000
39998,39998,"Just saw the movie, and the scary thing was, t...",1,0.009800,0.000000,0.0,0.0,0.009911,0.005896


In [10]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pektu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pektu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pektu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [11]:
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.porter import PorterStemmer
porter_stemmer = PorterStemmer()

en_stop = list(stopwords.words('english'))
lemmatizer = WordNetLemmatizer() 

def tokenize(text):
    tokens = nltk.word_tokenize(text.lower())
    tokens = [t for t in tokens if
              re.match(r'[^\W\d]*$', t) and (len(t) > 2) and (t not in en_stop)]
    tokens = [lemmatizer.lemmatize(t) for t in tokens]
    return tokens

tokens = df['review'].apply(tokenize)
#tokens = [simple_preprocess(remove_stopwords(review), deacc=True) for review in df['review']]

In [308]:
#tokens = [[porter_stemmer.stem(word) for word in review] for review in tokens]

In [12]:
model = Word2Vec(tokens, size=512, window=11, sample=1e-5, min_count=5, iter=11, negative=20, sg=1)

In [12]:
#чтобы долго не ждать, можно написать
model = Word2Vec.load("word2vec.model")

In [13]:
model.save("word2vec.model")

In [18]:
model.wv.similarity('movie', 'film')

<ipython-input-18-7b7eabcc4bf7>:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  model.wv.similarity('movie', 'film')


0.867677

In [13]:
import gensim
wv = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [14]:
model = wv

In [6]:
def preprocess_df(df):
    df['review'] = df['review'].apply(lambda s: s.replace('<br />', ' '))
    df['excl_number'] = tfidf_symb(df['review'], '!')
    df['q_number'] = tfidf_symb(df['review'], '?')
    #df['dot_number'] = tfidf_symb(df['review'], '.')
    df['neg_number'] = tfidf_neg(df['review'])
    df['pos_number'] = tfidf_pos(df['review'])
    df['comm_number'] = tfidf_symb(df['review'], ',')
    df['double_excl_number'] = tfidf_symb(df['review'], '!!')
    
    return df

In [15]:
def encode(list_of_tokens):
    x = np.array([model.wv[t] for t in list_of_tokens if t in model.wv.vocab])
    #return [np.min(x, axis=0), np.mean(x, axis=0), np.max(x, axis=0)]
    return np.mean(x, axis=0)

fts = np.array([encode(t) for t in tokens])
fts.shape

<ipython-input-15-53c10694d8ba>:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  x = np.array([model.wv[t] for t in list_of_tokens if t in model.wv.vocab])


(40000, 300)

In [44]:
fts = np.array([x.reshape(1, 512 * 3)[0] for x in fts])

In [7]:
import math
def get_neg(s):
    a = s.count('('); b = s.count(')')
    if a - b <= 0: return 0
    else: return (a - b) / len(s)
def get_pos(s):
    a = s.count('('); b = s.count(')')
    if b - a <= 0: return 0
    else: return (b - a) / len(s)

In [8]:
def tfidf_symb(texts, sym):
    idf = np.log((len(texts) + 1) / (np.sum(texts.apply(lambda s: 0 if s.find(sym) == -1 else 1)) + 1)) + 1
    res = []
    for text in texts:
        tf = text.count(sym) / len(text)
        res.append(tf * idf)
    return res
def tfidf_neg(texts):
    idf = np.log((len(texts) + 1) / (np.sum(texts.apply(lambda s: 0 if get_neg(s) == 0 else 1)) + 1)) + 1
    res = []
    for text in texts:
        tf = get_neg(text) / len(text)
        res.append(tf * idf)
    return res
def tfidf_pos(texts):
    idf = np.log((len(texts) + 1) / (np.sum(texts.apply(lambda s: 0 if get_pos(s) == 0 else 1)) + 1)) + 1
    res = []
    for text in texts:
        tf = get_pos(text) / len(text)
        res.append(tf * idf)
    return res

import re
def count_upper(s):
    return len(re.findall(r'[A-Z]',s))


def tfidf_big(texts):
    res = []
    for text in texts:
        n = count_upper(text)
        tf = n / len(text)
        idf = np.log((len(texts) + 1) / (np.sum(texts.apply(lambda s: 0 if count_upper(s) >= n else 1)) + 1)) + 1
        res.append(tf * idf)
    return res

In [19]:
def preprocess_fts(words, df):
    names = ['excl_number', 'double_excl_number', 'q_number', 'neg_number', 'pos_number', 'comm_number']
    for name in names:
        words = np.hstack([words, np.array(df[name]).reshape(df[name].shape[0], 1)])
    return words

In [20]:
fts = preprocess_fts(fts, df)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(fts, df.label.values,
                                                    test_size=0.2, shuffle=True, random_state = 11)

In [22]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(with_mean = False)
X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.transform(X_test)

In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
#param_grid = {'C': [0.01, 0.05, 0.1, 0.5, 1, 5, 10]}
#estimator = LogisticRegression(solver='lbfgs')
#cv = 3
#optimizer = GridSearchCV(estimator, param_grid, cv=cv, scoring='f1')
#optimizer.fit(X_train_sc, y_train)

#clf = DecisionTreeClassifier().fit(X_train_sc, y_train)
#clf_1 = LogisticRegression(solver='lbfgs', max_iter=5000).fit(X_train_sc, y_train)
#clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=7, random_state=0).fit(X_train, y_train)
#clf = RandomForestClassifier(n_estimators = 100, max_depth=11, random_state=0).fit(X_train_sc, y_train)
clf_1 = svm.SVC(kernel='rbf', probability=True, gamma='auto')
clf_1.fit(X_train_sc, y_train)
#clf_1 = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=11, max_iter=5, tol=None)
#clf_1.fit(X_train_sc, y_train)
#clf_1 = AdaBoostClassifier(n_estimators = 800).fit(X_train_sc, y_train)
#svm = svm.SVC(gamma='auto',probability=True)
#clf_1 = BaggingClassifier(base_estimator=svm, n_estimators=10, random_state=11, max_samples=2000, bootstrap = False, n_jobs=-1,
#                        warm_start=True)
#clf_1.fit(X_train_sc,y_train)

SVC(gamma='auto', probability=True)

In [32]:
y_pred_test = clf_1.predict(X_test_sc)
y_pred_train = clf_1.predict(X_train_sc)
print('Model test f1 score: {0:0.6f}'. format(f1_score(y_test, y_pred_test)))#0.891681      0.884116
print('Model train f1 score: {0:0.6f}'. format(f1_score(y_train, y_pred_train)))#0.924731   0.905931

Model test f1 score: 0.870960
Model train f1 score: 0.935294


In [ ]:
X_test

In [33]:
datat = 'test.csv'
df_test = pd.read_csv(datat)
df_test.head

<bound method NDFrame.head of       Unnamed: 0                                             review
0              0  The make -or-break of a love story for me is w...
1              1  "Bend It Like Beckham" is a film that got very...
2              2  Pete's Meteor. I seen this referred to as "aut...
3              3  Funny that I find myself forced to review this...
4              4  Bare Wench is another softcore parody of the B...
...          ...                                                ...
9995        9995  The only reason I wanted to see this was becau...
9996        9996  Three Russian aristocrats soak up the decadenc...
9997        9997  Greetings again from the darkness. Remember al...
9998        9998  This film is a tour de force from Julie Taymor...
9999        9999  I'll be honest,I finally checked this movie no...

[10000 rows x 2 columns]>

In [34]:
preprocess_df(df_test)

,Unnamed: 0,review,excl_number,q_number,neg_number,pos_number,comm_number,double_excl_number
0,0,The make -or-break of a love story for me is w...,0.001547,0.000000,0.0,0.0,0.005447,0.000000
1,1,"""Bend It Like Beckham"" is a film that got very...",0.002798,0.000000,0.0,0.0,0.007036,0.000000
2,2,"Pete's Meteor. I seen this referred to as ""aut...",0.000000,0.000867,0.0,0.0,0.007723,0.000000
3,3,Funny that I find myself forced to review this...,0.000000,0.000474,0.0,0.0,0.018935,0.000000
4,4,Bare Wench is another softcore parody of the B...,0.009982,0.000000,0.0,0.0,0.010039,0.000000
...,...,...,...,...,...,...,...,...
9995,9995,The only reason I wanted to see this was becau...,0.000000,0.000000,0.0,0.0,0.011496,0.000000
9996,9996,Three Russian aristocrats soak up the decadenc...,0.000000,0.000000,0.0,0.0,0.019489,0.000000
9997,9997,Greetings again from the darkness. Remember al...,0.003582,0.001871,0.0,0.0,0.008106,0.003242
9998,9998,This film is a tour de force from Julie Taymor...,0.000000,0.000000,0.0,0.0,0.004883,0.000000


In [35]:
tokens_test = df_test['review'].apply(tokenize)

In [36]:
fts_test = np.array([encode(t) for t in tokens_test])
fts_test.shape

<ipython-input-15-53c10694d8ba>:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  x = np.array([model.wv[t] for t in list_of_tokens if t in model.wv.vocab])


(10000, 300)

In [69]:
fts_test = np.array([x.reshape(1, 512 * 3)[0] for x in fts_test])
fts_test.shape

ValueError: cannot reshape array of size 512 into shape (1,1536)

In [37]:
fts_test = preprocess_fts(fts_test, df_test)
fts_test.shape

(10000, 306)

In [38]:
X_all = scaler.fit_transform(fts)
fts_test_sc = scaler.transform(fts_test)

In [39]:
clf_1.fit(X_all, df['label'].values)

SVC(gamma='auto', probability=True)

In [40]:
from sklearn.metrics import classification_report
print(classification_report(df['label'].values, clf_1.predict(X_all)))
print('Model f1 score: {0:0.6f}'. format(f1_score(df['label'].values, clf_1.predict(X_all))))

              precision    recall  f1-score   support

           0       0.94      0.93      0.93     20067
           1       0.93      0.94      0.93     19933

    accuracy                           0.93     40000
   macro avg       0.93      0.93      0.93     40000
weighted avg       0.93      0.93      0.93     40000

Model f1 score: 0.934020


In [41]:
res = clf_1.predict(fts_test_sc)
ans = pd.DataFrame(np.array([df_test['Unnamed: 0'], res]).T, columns=[['Id', 'Predicted']])
ans.to_csv('Gason_task_4.csv', index = False)